In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from Bio import SeqIO
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel, T5Tokenizer, T5EncoderModel
from tqdm import tqdm
import logging
from pathlib import Path
import re
import os, json, math, time, sys
from typing import Dict, Tuple, List
import scipy.sparse as sp
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_recall_curve
from tqdm import tqdm
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch.serialization as _ser
_ser.add_safe_globals([np.core.multiarray.scalar])

state = torch.load(last_ckpt, map_location=DEVICE, weights_only=False)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device =", DEVICE)
# Load ProtT5 XL UniRef50 Model + Tokenizer
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc", do_lower_case=False)
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc")
model = model.to(DEVICE)
model.eval()

print("[OK] ProtT5 model + tokenizer loaded.")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("Using device:", DEVICE)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Device = cuda
[OK] ProtT5 model + tokenizer loaded.
Using device: cuda
/kaggle/input/cafa-6-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-6-protein-function-prediction/IA.tsv
/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta
/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset-taxon-list.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-6-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo


In [22]:
import torch, transformers, google.protobuf
print("torch.cuda:", torch.cuda.is_available(), torch.cuda.get_device_name(0) if torch.cuda.is_available() else "no-gpu")
print("torch.__version__:", torch.__version__)
print("transformers:", transformers.__version__)
print("protobuf:", google.protobuf.__version__)

# quick import check to ensure no JAX/XLA interference:
try:
    import jax
    print("JAX detected:", jax.__version__)
except Exception as e:
    print("JAX not present or import failed (OK if you don't use TPU):", e)


torch.cuda: True Tesla T4
torch.__version__: 2.6.0+cu124
transformers: 4.53.3
protobuf: 6.33.0
JAX detected: 0.5.2


In [23]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
logger = logging.getLogger(__name__)

# ===========================================================
# FORCE GPU
# ===========================================================
assert torch.cuda.is_available(), "❌ GPU NOT AVAILABLE — KAGGLE RUNNING ON CPU!"

DEVICE = torch.device("cuda")
logger.info(f"🔥 GPU ACTIVE: {torch.cuda.get_device_name(0)}")


# ===========================================================
# FASTA LOADER
# ===========================================================
def load_fasta(path):
    ids, seqs = [], []
    with open(path, "r") as f:
        cur = None
        buf = []
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                if cur:
                    seqs.append("".join(buf))
                cur = line[1:]
                ids.append(cur)
                buf = []
            else:
                buf.append(line)
        seqs.append("".join(buf))
    return ids, seqs



# ===========================================================
# GPU-FORCED PROTBERT EMBEDDER
# ===========================================================
class ProtBertEmbedder:
    def __init__(self, model_name="Rostlab/prot_bert_bfd"):

        logger.info(f"Loading ProtBERT from HuggingFace → GPU")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False)
        self.model = AutoModel.from_pretrained(model_name)
        self.model = self.model.to(DEVICE)
        self.model.eval()

        logger.info("Model successfully moved to GPU ✔")

    def embed(self, sequences, batch_size=16):
        logger.info(f"Embedding {len(sequences)} sequences using GPU")

        seqs = [re.sub("[UZOB]", "X", s) for s in sequences]
        embeddings = []

        # GPU autocast for speed + memory reduction
        autocast = torch.cuda.amp.autocast

        with torch.inference_mode():
            for i in tqdm(range(0, len(seqs), batch_size), desc="GPU-ProtBERT"):

                batch = seqs[i : i + batch_size]

                # --- TOKENIZATION (CPU → GPU explicitly) ---
                toks = self.tokenizer(
                    batch,
                    add_special_tokens=True,
                    padding=True,
                    truncation=True,
                    return_tensors="pt"
                )

                # Force token tensors to GPU
                toks = {k: v.to(DEVICE) for k, v in toks.items()}

                # --- MODEL FORWARD PASS ON GPU ---
                with autocast(dtype=torch.float16):
                    out = self.model(**toks)

                hidden = out.last_hidden_state     # (B, L, 1024)
                mask = toks["attention_mask"]

                # --- MEAN POOLING ---
                for j in range(hidden.size(0)):
                    L = mask[j].sum()
                    emb = hidden[j, :L].mean(dim=0)
                    embeddings.append(emb.float().cpu().numpy())

        return np.vstack(embeddings)



# ===========================================================
# MAIN SCRIPT
# ===========================================================
TRAIN_FASTA = Path("/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta")
TEST_FASTA  = Path("/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta")

OUTPUT = Path("/kaggle/working/embeddings")
OUTPUT.mkdir(exist_ok=True)

logger.info("Loading FASTA...")

train_ids, train_seqs = load_fasta(TRAIN_FASTA)
test_ids, test_seqs   = load_fasta(TEST_FASTA)

train_sequences = dict(zip(train_ids, train_seqs))
test_sequences = dict(zip(test_ids, test_seqs))

embedder = ProtBertEmbedder()

# ========================= TRAIN =============================
logger.info("🔥 Using GPU to embed TRAIN sequences...")
train_embs = []
BATCH = 16

for i in range(0, len(train_ids), BATCH):
    batch_ids = train_ids[i : i + BATCH]
    batch_seqs = [train_sequences[x] for x in batch_ids]

    batch_embs = embedder.embed(batch_seqs, batch_size=BATCH)
    train_embs.append(batch_embs)

train_embs = np.vstack(train_embs)
np.save(OUTPUT / "train_protbert_gpu.npy", train_embs)
logger.info(f"[OK] Saved TRAIN embeddings: {train_embs.shape}")


# ========================= TEST ==============================
logger.info("🔥 Using GPU to embed TEST sequences...")

test_embs = []

for i in range(0, len(test_ids), BATCH):
    batch_ids = test_ids[i : i + BATCH]
    batch_seqs = [test_sequences[x] for x in batch_ids]

    batch_embs = embedder.embed(batch_seqs, batch_size=BATCH)
    test_embs.append(batch_embs)

test_embs = np.vstack(test_embs)
np.save(OUTPUT / "test_protbert_gpu.npy", test_embs)
logger.info(f"[OK] Saved TEST embeddings: {test_embs.shape}")

logger.info("=== GPU EMBEDDING COMPLETE ===")

GPU-ProtBERT:   0%|          | 0/1 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/tmp/ipykernel_47/2648483844.py:77: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=torch.float16):
GPU-ProtBERT:   0%|          | 0/1 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [24]:
wrong_df = pd.read_csv("/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv", sep="\t", header=None)
print(wrong_df.head(20))

          0           1       2
0   EntryID        term  aspect
1    Q5W0B1  GO:0000785       C
2    Q5W0B1  GO:0004842       F
3    Q5W0B1  GO:0051865       P
4    Q5W0B1  GO:0006275       P
5    Q5W0B1  GO:0006513       P
6    Q5W0B1  GO:0003682       F
7    Q5W0B1  GO:0005515       F
8    Q3EC77  GO:0000138       C
9    Q3EC77  GO:0005794       C
10   Q8IZR5  GO:0005515       F
11   Q8R2Z3  GO:0140900       F
12   Q8R2Z3  GO:0005254       F
13   Q8R2Z3  GO:0035429       P
14   Q8R2Z3  GO:0016323       C
15   Q8R2Z3  GO:1902358       P
16   Q8R2Z3  GO:0015701       P
17   Q8R2Z3  GO:0015705       P
18   Q8R2Z3  GO:0015706       P
19   Q8R2Z3  GO:0019532       P


In [41]:
# 02_build_go_label_matrix.py
import os
import json
from pathlib import Path
from collections import defaultdict, deque
from scipy.sparse import csr_matrix, save_npz

# ---------- CONFIG ----------
BASE = Path("/kaggle/working")               # change if local
DATA_DIR = Path("/kaggle/input/cafa-6-protein-function-prediction/")
TRAIN_TERMS = DATA_DIR / "Train" / "train_terms.tsv"
GO_OBO = DATA_DIR / "Train" / "go-basic.obo"
OUT = BASE / "outputs_labels"
OUT.mkdir(parents=True, exist_ok=True)

# ---------- HELPERS ----------
def parse_obo(path):
    """
    Parse go-basic.obo to extract:
        term -> {'id': GO:xxxx, 'namespace': MF/BP/CC, 'is_a': [parents]}
    Returns dict of id -> node.
    """
    nodes = {}
    cur = None
    for line in open(path, "r", encoding="utf-8"):
        line = line.strip()
        if line == "[Term]":
            cur = {}
            continue
        if line == "" and cur and "id" in cur:
            nodes[cur["id"]] = cur
            cur = None
            continue
        if cur is None:
            continue
        if line.startswith("id: "):
            cur["id"] = line.split("id: ")[1].strip()
        elif line.startswith("namespace: "):
            cur["namespace"] = line.split("namespace: ")[1].strip()
        elif line.startswith("is_a: "):
            parent = line.split("is_a: ")[1].split(" ! ")[0].strip()
            cur.setdefault("is_a", []).append(parent)
    # last term
    if cur and "id" in cur:
        nodes[cur["id"]] = cur
    return nodes

def build_ancestors(nodes):
    """
    Build ancestor sets for every term via BFS up the DAG.
    Returns dict term -> set(ancestors incl. self).
    """
    ancestors = {}
    # prefill missing parents
    for tid, node in nodes.items():
        node.setdefault("is_a", [])
    for tid in nodes:
        # BFS/DFS up
        stack = [tid]
        seen = set()
        while stack:
            t = stack.pop()
            if t in seen:
                continue
            seen.add(t)
            parents = nodes.get(t, {}).get("is_a", [])
            for p in parents:
                if p and p not in seen:
                    stack.append(p)
        ancestors[tid] = seen  # includes self
    return ancestors

# ---------- LOAD TRAIN TERMS ----------
print("[*] Loading train terms...")
train_df = pd.read_csv(TRAIN_TERMS, sep="\t")
train_df.columns = ["protein", "go", "aspect"]

print(f"[*] Loaded {len(train_df)} annotations; unique proteins: {train_df['protein'].nunique()}")

# ---------- LOAD GO OBO ----------
print("[*] Parsing GO OBO...")
nodes = parse_obo(str(GO_OBO))
print(f"[*] Parsed {len(nodes)} GO nodes from OBO")

anc = build_ancestors(nodes)
print("[*] Built ancestors for GO terms")

# ---------- Filter GO terms present in OBO ----------
all_terms_in_train = set(train_df["go"].unique())
present_terms = sorted(t for t in all_terms_in_train if t in nodes)
missing_terms = sorted(list(all_terms_in_train - set(present_terms)))
print(f"[*] Terms in train: {len(all_terms_in_train)}; present in OBO: {len(present_terms)}; missing: {len(missing_terms)}")
if missing_terms:
    print("Sample missing terms:", missing_terms[:10])

# ---------- Partition terms by namespace ----------
namespace_terms = defaultdict(list)
for t in present_terms:
    ns = nodes[t].get("namespace", "unknown")
    namespace_terms[ns].append(t)

for ns in ["molecular_function", "biological_process", "cellular_component"]:
    print(f"[*] {ns}: {len(namespace_terms.get(ns,[]))} terms")

# ---------- Build term -> index maps per subontology ----------
maps = {}
for ns in ["molecular_function", "biological_process", "cellular_component"]:
    terms = sorted(namespace_terms.get(ns, []))
    maps[ns] = {"terms": terms, "index": {t: i for i, t in enumerate(terms)}}
    print(f"[*] {ns} indices: {len(terms)}")

# ---------- Build protein list and mapping ----------
proteins = sorted(train_df["protein"].unique())
protein_index = {p: i for i, p in enumerate(proteins)}
print(f"[*] Proteins count: {len(proteins)}")

# ---------- Build sparse matrices ----------
def build_sparse_for_ns(ns):
    terms = maps[ns]["terms"]
    term_idx = maps[ns]["index"]
    rows = []
    cols = []
    data = []
    for prot, group in train_df.groupby("protein"):
        pid = protein_index[prot]
        gos = set(group["go"].tolist())
        # keep only gos that belong to this namespace and are present
        gos = {g for g in gos if g in term_idx}
        if not gos:
            continue
        # expand to ancestors, filter to namespace
        expanded = set()
        for g in gos:
            expanded |= anc.get(g, set())
        expanded = {e for e in expanded if e in term_idx}
        for g in expanded:
            rows.append(pid)
            cols.append(term_idx[g])
            data.append(1)
    if not rows:
        print(f"[!] No annotations for {ns}")
        return None
    mat = csr_matrix((data, (rows, cols)), shape=(len(proteins), len(terms)), dtype=np.uint8)
    return mat

# build and save per-namespace
for ns_label, out_name in [("molecular_function", "labels_MF.npz"),
                           ("biological_process", "labels_BP.npz"),
                           ("cellular_component", "labels_CC.npz")]:
    print(f"[*] Building matrix for {ns_label} ...")
    mat = build_sparse_for_ns(ns_label)
    if mat is None:
        print(f"[!] {ns_label} empty, skipping save.")
        continue
    save_path = OUT / out_name
    save_npz(save_path, mat)
    print(f"[OK] Saved {ns_label} matrix -> {save_path}, shape = {mat.shape}, nnz={mat.nnz}")

# ---------- Save auxiliary maps ----------
maps_out = {
    "protein_index": {p: i for p, i in protein_index.items()},
    "mf_terms": maps["molecular_function"]["terms"],
    "bp_terms": maps["biological_process"]["terms"],
    "cc_terms": maps["cellular_component"]["terms"]
}
with open(OUT / "maps.json", "w") as fh:
    json.dump(maps_out, fh)

print("[*] All done. Label matrices saved to:", OUT)

anc = build_ancestors(nodes)
print("[*] Built ancestors for GO terms")
# Save ancestors for inference
with open(OUT / "ancestors.json", "w") as f:
    json.dump({k: list(v) for k, v in anc.items()}, f)
print("[OK] Saved ancestors.json")



[*] Loading train terms...
[*] Loaded 537027 annotations; unique proteins: 82404
[*] Parsing GO OBO...
[*] Parsed 48101 GO nodes from OBO
[*] Built ancestors for GO terms
[*] Terms in train: 26125; present in OBO: 26125; missing: 0
[*] molecular_function: 6616 terms
[*] biological_process: 16858 terms
[*] cellular_component: 2651 terms
[*] molecular_function indices: 6616
[*] biological_process indices: 16858
[*] cellular_component indices: 2651
[*] Proteins count: 82404
[*] Building matrix for molecular_function ...
[OK] Saved molecular_function matrix -> /kaggle/working/outputs_labels/labels_MF.npz, shape = (82404, 6616), nnz=427243
[*] Building matrix for biological_process ...
[OK] Saved biological_process matrix -> /kaggle/working/outputs_labels/labels_BP.npz, shape = (82404, 16858), nnz=1270993
[*] Building matrix for cellular_component ...
[OK] Saved cellular_component matrix -> /kaggle/working/outputs_labels/labels_CC.npz, shape = (82404, 2651), nnz=403670
[*] All done. Label m

In [29]:
import torch.serialization as _ser

_ser.add_safe_globals([
    np.dtype,
    np.core.multiarray.scalar
])


In [30]:
state = torch.load(last_ckpt, map_location=DEVICE, weights_only=False)


In [32]:
# ------------------------
# CONFIG
# ------------------------
BASE = Path("/kaggle/working")
EMB_PATH = BASE / "embeddings" / "train_protbert_gpu.npy"
LABEL_DIR = BASE / "outputs_labels"
MODEL_DIR = BASE / "models"
MODEL_DIR.mkdir(exist_ok=True)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42

# training hyperparams - adjust if you need
BATCH_SIZE = 128         # increase/decrease per GPU memory
EPOCHS = 10
LR = 1e-4
WEIGHT_DECAY = 1e-5
PATIENCE = 8             # early stopping on val F1 (mean of three heads)
PROJ_DIM = 512           # backbone projection dim
HID = 1024
DROPOUT = 0.3
NUM_WORKERS = 4

np.random.seed(SEED)
torch.manual_seed(SEED)

print("DEVICE:", DEVICE)
# ------------------------
# HELPERS: load embeddings + labels
# ------------------------
def load_embeddings(path: Path) -> np.ndarray:
    print("Loading embeddings (memmap)...", path)
    arr = np.load(path, mmap_mode="r")
    print("Embeddings shape:", arr.shape)
    return arr

def load_label_npz(path: Path) -> sp.csr_matrix:
    print("Loading label npz:", path)
    return sp.load_npz(str(path)).tocsr()

# ------------------------
# Dataset
# ------------------------
class EmbLabelDataset(Dataset):
    def __init__(self, emb_mem: np.ndarray, label_matrices: Dict[str, sp.csr_matrix], indices: List[int]):
        """
        emb_mem: memory-mapped numpy array (n_proteins, emb_dim)
        label_matrices: dict of 'mf','bp','cc' -> csr_matrix (n_proteins x n_terms)
        indices: list of integer protein indices to include (train/val split)
        """
        self.emb = emb_mem
        self.labels = label_matrices
        self.indices = indices

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        i = self.indices[idx]
        x = np.array(self.emb[i], dtype=np.float32)           # (D,)
        # get dense per-ontology label vectors (convert sparse row -> dense)
        y_mf = self.labels['mf'][i].toarray().ravel().astype(np.uint8) if self.labels['mf'].shape[0] > 0 else np.zeros(0, dtype=np.uint8)
        y_bp = self.labels['bp'][i].toarray().ravel().astype(np.uint8) if self.labels['bp'].shape[0] > 0 else np.zeros(0, dtype=np.uint8)
        y_cc = self.labels['cc'][i].toarray().ravel().astype(np.uint8) if self.labels['cc'].shape[0] > 0 else np.zeros(0, dtype=np.uint8)
        return x, y_mf, y_bp, y_cc

def collate_fn(batch):
    X = torch.tensor(np.stack([b[0] for b in batch]), dtype=torch.float32)
    Ym = [b[1] for b in batch]
    Yb = [b[2] for b in batch]
    Yc = [b[3] for b in batch]
    # Convert lists of arrays -> tensors with appropriate shapes
    Ym = torch.tensor(np.vstack(Ym), dtype=torch.float32) if Ym[0].size else torch.empty((len(batch),0), dtype=torch.float32)
    Yb = torch.tensor(np.vstack(Yb), dtype=torch.float32) if Yb[0].size else torch.empty((len(batch),0), dtype=torch.float32)
    Yc = torch.tensor(np.vstack(Yc), dtype=torch.float32) if Yc[0].size else torch.empty((len(batch),0), dtype=torch.float32)
    return X, Ym, Yb, Yc

# ------------------------
# Model
# ------------------------
class MultiHeadNet(nn.Module):
    def __init__(self, emb_dim:int, proj_dim:int, hid:int, n_mf:int, n_bp:int, n_cc:int, dropout=float(0.3)):
        super().__init__()
        # shared projection
        self.proj = nn.Sequential(
            nn.LayerNorm(emb_dim),
            nn.Linear(emb_dim, proj_dim),
            nn.GELU(),
            nn.Dropout(dropout)
        )
        # common trunk
        self.trunk = nn.Sequential(
            nn.Linear(proj_dim, hid),
            nn.LayerNorm(hid),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hid, proj_dim),
            nn.GELU(),
            nn.Dropout(dropout)
        )
        # heads
        self.head_mf = nn.Linear(proj_dim, n_mf) if n_mf>0 else None
        self.head_bp = nn.Linear(proj_dim, n_bp) if n_bp>0 else None
        self.head_cc = nn.Linear(proj_dim, n_cc) if n_cc>0 else None

    def forward(self, x):
        x = self.proj(x)
        x = self.trunk(x)
        out_mf = self.head_mf(x) if self.head_mf is not None else None
        out_bp = self.head_bp(x) if self.head_bp is not None else None
        out_cc = self.head_cc(x) if self.head_cc is not None else None
        return out_mf, out_bp, out_cc

# ------------------------
# Loss helpers (per head)
# ------------------------
def make_loss(outputs, targets):
    # outputs, targets are tensors (B, C) or None
    loss = 0.0
    count = 0
    criterion = nn.BCEWithLogitsLoss()
    for out, tgt in zip(outputs, targets):
        if out is None or tgt.numel()==0:
            continue
        loss = loss + criterion(out, tgt)
        count += 1
    return loss if count>0 else torch.tensor(0.0, device=DEVICE)

# ------------------------
# Metrics: simple per-ontology F1 (macro average)
# ------------------------
def threshold_search(y_true: np.ndarray, y_score: np.ndarray):
    """
    For each class, find best threshold on val set using F1 from precision_recall_curve.
    Returns vector of thresholds (len = n_classes).
    """
    n_classes = y_true.shape[1]
    thrs = np.zeros(n_classes, dtype=float)
    for j in range(n_classes):
        yj = y_true[:, j]
        sj = y_score[:, j]
        if yj.sum() == 0:
            thrs[j] = 0.5
            continue
        precision, recall, thr = precision_recall_curve(yj, sj)
        f1 = (2*precision*recall) / (precision+recall+1e-12)
        idx = np.nanargmax(f1)
        # thr array len = len(precision)-1; handle boundaries:
        if idx >= len(thr):
            chosen = 0.5
        else:
            chosen = thr[idx]
        thrs[j] = float(chosen)
    return thrs

def compute_f1_at_thresholds(y_true, y_score, thresholds):
    y_pred = (y_score >= thresholds).astype(int)
    # per-class f1 -> mean
    f1s = []
    for j in range(y_true.shape[1]):
        if y_true[:, j].sum()==0:
            continue
        f1s.append(f1_score(y_true[:, j], y_pred[:, j], zero_division=0))
    if len(f1s)==0:
        return 0.0
    return float(np.mean(f1s))

# ------------------------
# Load data
# ------------------------
print("Loading embeddings and label matrices...")
emb = load_embeddings(EMB_PATH)   # memmap (N, D)
lab_mf = load_label_npz(LABEL_DIR / "labels_MF.npz")
lab_bp = load_label_npz(LABEL_DIR / "labels_BP.npz")
lab_cc = load_label_npz(LABEL_DIR / "labels_CC.npz")

n_proteins, emb_dim = emb.shape
n_mf = lab_mf.shape[1]
n_bp = lab_bp.shape[1]
n_cc = lab_cc.shape[1]
print(f"n_prot={n_proteins}, emb_dim={emb_dim}, MF={n_mf}, BP={n_bp}, CC={n_cc}")

# ------------------------
# Train/Val split (stratified by number of labels per protein)
# ------------------------
all_idx = np.arange(n_proteins)
# stratify by label counts (sum MF+BP+CC)
label_counts = (
    lab_mf.sum(axis=1).A1 +
    lab_bp.sum(axis=1).A1 +
    lab_cc.sum(axis=1).A1
).astype(int)

# create bins
bins = np.minimum(10, (label_counts // 5) + 1)
train_idx, val_idx = train_test_split(all_idx, test_size=0.15, random_state=SEED, stratify=bins)

print("Train size:", len(train_idx), "Val size:", len(val_idx))

# ------------------------
# Datasets & Loaders
# ------------------------
label_mats = {'mf': lab_mf, 'bp': lab_bp, 'cc': lab_cc}
train_ds = EmbLabelDataset(emb, label_mats, train_idx.tolist())
val_ds   = EmbLabelDataset(emb, label_mats, val_idx.tolist())

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, collate_fn=collate_fn)

# ------------------------
# Model init
# ------------------------
model = MultiHeadNet(emb_dim=emb_dim, proj_dim=PROJ_DIM, hid=HID, n_mf=n_mf, n_bp=n_bp, n_cc=n_cc, dropout=DROPOUT)
model = model.to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type=="cuda"))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

best_val = -1.0
pat = 0

# ------------------------
# Training loop
# ------------------------
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0.0
    t0 = time.time()
    for X, Ym, Yb, Yc in tqdm(train_loader, desc=f"Train E{epoch}"):
        X = X.to(DEVICE)
        Ym = Ym.to(DEVICE) if Ym.numel() else Ym.to(DEVICE)
        Yb = Yb.to(DEVICE) if Yb.numel() else Yb.to(DEVICE)
        Yc = Yc.to(DEVICE) if Yc.numel() else Yc.to(DEVICE)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=(DEVICE.type=="cuda")):
            out_mf, out_bp, out_cc = model(X)
            loss = make_loss((out_mf, out_bp, out_cc), (Ym, Yb, Yc))
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        epoch_loss += float(loss.detach().cpu().numpy())

    scheduler.step()
    t1 = time.time()

    # ------------------------
    # Validation: collect scores
    # ------------------------
    model.eval()
    all_scores = {'mf': [], 'bp': [], 'cc': []}
    all_trues  = {'mf': [], 'bp': [], 'cc': []}

    with torch.inference_mode():
        for X, Ym, Yb, Yc in tqdm(val_loader, desc=f"Val E{epoch}"):
            X = X.to(DEVICE)
            out_mf, out_bp, out_cc = model(X)
            # move to cpu numpy
            if out_mf is not None:
                all_scores['mf'].append(out_mf.sigmoid().cpu().numpy())
                all_trues ['mf'].append(Ym.cpu().numpy())
            if out_bp is not None:
                all_scores['bp'].append(out_bp.sigmoid().cpu().numpy())
                all_trues ['bp'].append(Yb.cpu().numpy())
            if out_cc is not None:
                all_scores['cc'].append(out_cc.sigmoid().cpu().numpy())
                all_trues ['cc'].append(Yc.cpu().numpy())

    # stack arrays
    val_scores = {}
    val_trues = {}
    for k in all_scores:
        val_scores[k] = np.vstack(all_scores[k]) if len(all_scores[k])>0 else np.zeros((0,0))
        val_trues[k]  = np.vstack(all_trues[k])  if len(all_trues[k])>0 else np.zeros((0,0))

    # ------------------------
    # Threshold tuning & F1 eval (mean-of-heads)
    # ------------------------
    val_f1s = {}
    val_thrs = {}
    for k in ['mf','bp','cc']:
        if val_trues[k].size==0:
            val_f1s[k] = 0.0
            val_thrs[k] = np.array([])
            continue
        # compute per-class thresholds on val (expensive but only once per epoch)
        th = threshold_search(val_trues[k], val_scores[k])
        val_thrs[k] = th
        f1 = compute_f1_at_thresholds(val_trues[k], val_scores[k], th)
        val_f1s[k] = f1

    mean_val = np.mean([val_f1s['mf'], val_f1s['bp'], val_f1s['cc']])
    print(f"E{epoch} loss={epoch_loss:.4f} time={t1-t0:.1f}s MF_f1={val_f1s['mf']:.4f} BP_f1={val_f1s['bp']:.4f} CC_f1={val_f1s['cc']:.4f} mean={mean_val:.4f}")

    # ------------------------
    # checkpointing
    # ------------------------
    if mean_val > best_val:
        best_val = mean_val
        pat = 0
        ckpt = MODEL_DIR / f"multihead_epoch{epoch}_mean{mean_val:.4f}.pth"
        torch.save({
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "epoch": epoch,
            "mean_val": mean_val,
            "thresholds": {k: val_thrs[k].tolist() for k in val_thrs}
        }, ckpt)
        print("[OK] Saved checkpoint:", ckpt)
    else:
        pat += 1
        print("[..] no improvement pat=", pat)
        if pat >= PATIENCE:
            print("[!!] Early stopping triggered.")
            break

# ------------------------
# Save final thresholds (from best checkpoint if exists)

ckpt_files = sorted(MODEL_DIR.glob("multihead_epoch*.pth"))
if ckpt_files:
    last_ckpt = ckpt_files[-1]
    _ser.add_safe_globals([np.dtype, _np.core.multiarray.scalar])

    state = torch.load(last_ckpt, map_location=DEVICE, weights_only=False)
    thr_out = state.get("thresholds", {})
else:
    thr_out = {k: (val_thrs[k].tolist() if k in val_thrs else []) for k in ['mf','bp','cc']}

with open(MODEL_DIR / "thresholds.json", "w") as fh:
    json.dump({k: (thr_out.get(k, [])) for k in thr_out}, fh)



DEVICE: cuda
Loading embeddings and label matrices...
Loading embeddings (memmap)... /kaggle/working/embeddings/train_protbert_gpu.npy
Embeddings shape: (82404, 1024)
Loading label npz: /kaggle/working/outputs_labels/labels_MF.npz
Loading label npz: /kaggle/working/outputs_labels/labels_BP.npz
Loading label npz: /kaggle/working/outputs_labels/labels_CC.npz
n_prot=82404, emb_dim=1024, MF=6616, BP=16858, CC=2651
Train size: 70043 Val size: 12361


/tmp/ipykernel_47/2313830678.py:215: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type=="cuda"))
Train E0:   0%|          | 0/548 [00:00<?, ?it/s]/tmp/ipykernel_47/2313830678.py:235: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE.type=="cuda")):
Val E0: 100%|██████████| 97/97 [00:03<00:00, 29.22it/s]


E0 loss=67.0798 time=14.8s MF_f1=0.0029 BP_f1=0.0030 CC_f1=0.0054 mean=0.0038
[OK] Saved checkpoint: /kaggle/working/models/multihead_epoch0_mean0.0038.pth


Train E1:   0%|          | 0/548 [00:00<?, ?it/s]/tmp/ipykernel_47/2313830678.py:235: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE.type=="cuda")):
Val E1: 100%|██████████| 97/97 [00:03<00:00, 29.26it/s]


E1 loss=10.3747 time=14.3s MF_f1=0.0028 BP_f1=0.0029 CC_f1=0.0052 mean=0.0036
[..] no improvement pat= 1


...

E7 loss=9.6575 time=14.2s MF_f1=0.0028 BP_f1=0.0029 CC_f1=0.0052 mean=0.0037
[..] no improvement pat= 7


Train E8:   0%|          | 0/548 [00:00<?, ?it/s]/tmp/ipykernel_47/2313830678.py:235: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE.type=="cuda")):
Val E8: 100%|██████████| 97/97 [00:03<00:00, 29.26it/s]


E8 loss=9.6410 time=14.2s MF_f1=0.0030 BP_f1=0.0030 CC_f1=0.0055 mean=0.0038
[OK] Saved checkpoint: /kaggle/working/models/multihead_epoch8_mean0.0038.pth


Train E9:   0%|          | 0/548 [00:00<?, ?it/s]/tmp/ipykernel_47/2313830678.py:235: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE.type=="cuda")):
Val E9: 100%|██████████| 97/97 [00:03<00:00, 29.27it/s]


E9 loss=9.6300 time=14.2s MF_f1=0.0028 BP_f1=0.0029 CC_f1=0.0052 mean=0.0036
[..] no improvement pat= 1


In [37]:
# Add safe globals if needed
import torch.serialization as _ser
_ser.add_safe_globals([np.dtype, np.core.multiarray.scalar])

# Load checkpoint safely
state = torch.load(last_ckpt, map_location=DEVICE, weights_only=False)

# Restore weights
model.load_state_dict(state["model_state"])

# Restore thresholds (used for converting probabilities to GO predictions)
thr_out = state["thresholds"]

print("Checkpoint restored from:", last_ckpt)
print("Epoch:", state["epoch"])
print("Best validation mean:", state["mean_val"])


Checkpoint restored from: /kaggle/working/models/multihead_epoch8_mean0.0038.pth
Epoch: 8
Best validation mean: 0.0038362937881450732


In [45]:
model = MultiHeadModel(
    input_dim=1024,
    proj_dim=512,
    mf_dim=len(mf_terms),
    bp_dim=len(bp_terms),
    cc_dim=len(cc_terms)
).to(DEVICE)


In [51]:
import torch
import torch.nn as nn

class MultiHeadModel(nn.Module):
    def __init__(self, input_dim=1024, proj_dim=512, hid=1024,
                 mf_dim=6616, bp_dim=16858, cc_dim=2651,
                 dropout=0.2):
        super().__init__()

        # Projection matches proj.0 and proj.1 in checkpoint
        self.proj = nn.Sequential(
            nn.LayerNorm(input_dim),            # proj.0
            nn.Linear(input_dim, proj_dim),     # proj.1
        )

        # Trunk matches trunk.0, trunk.1, trunk.4 in checkpoint
        self.trunk = nn.Sequential(
            nn.Linear(proj_dim, hid),           # trunk.0
            nn.LayerNorm(hid),                  # trunk.1
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hid, proj_dim),           # trunk.4
            nn.GELU(),
            nn.Dropout(dropout),
        )

        # Output heads EXACTLY matching checkpoint shapes
        self.head_mf = nn.Linear(proj_dim, mf_dim)
        self.head_bp = nn.Linear(proj_dim, bp_dim)
        self.head_cc = nn.Linear(proj_dim, cc_dim)

    def forward(self, x):
        x = self.proj(x)
        x = self.trunk(x)
        mf = self.head_mf(x)
        bp = self.head_bp(x)
        cc = self.head_cc(x)
        return mf, bp, cc


In [52]:
BASE = Path("/kaggle/working")
EMB_TEST = BASE / "embeddings" / "test_protbert_gpu.npy"
LABEL_DIR = BASE / "outputs_labels"
MODEL_DIR = BASE / "models"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===============================
# LOAD LABEL MAPS & ANCESTORS
# ===============================
maps = json.load(open(LABEL_DIR / "maps.json"))

mf_terms = maps["mf_terms"]
bp_terms = maps["bp_terms"]
cc_terms = maps["cc_terms"]

# Load ancestors for propagation
def load_ancestors():
    # reuse the build_ancestors function output saved earlier
    anc_path = LABEL_DIR / "ancestors.json"
    if anc_path.exists():
        return json.load(open(anc_path))
    else:
        raise FileNotFoundError("ancestors.json missing! Re-run OBO parser.")

ancestors = load_ancestors()

# ===============================
# LOAD MODEL
# ===============================
ckpt_files = sorted(list(MODEL_DIR.glob("*.pth")))
assert len(ckpt_files) > 0, "No model checkpoint found!"
last_ckpt = ckpt_files[-1]
print("Loading:", last_ckpt)

state = torch.load(last_ckpt, map_location=DEVICE, weights_only=False)

model = MultiHeadModel(
    input_dim=1024,
    proj_dim=512,
    mf_dim=len(mf_terms),
    bp_dim=len(bp_terms),
    cc_dim=len(cc_terms)
).to(DEVICE)

model.load_state_dict(state["model_state"])
thresholds = state["thresholds"]

model.eval()
print("[OK] Model loaded.")


# ===============================
# LOAD TEST EMBEDDINGS
# ===============================
print("Loading test embeddings...")
test_emb = np.load(EMB_TEST, mmap_mode='r')
print("Test emb shape:", test_emb.shape)

# ===============================
# TEST IDS
# ===============================
TEST_FASTA = Path("/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset-taxon-list.tsv")
test_ids_df = pd.read_csv(TEST_FASTA, sep="\t", header=None, names=["id", "tax"])
test_ids = test_ids_df["id"].tolist()

print("Test IDs:", len(test_ids))


# ===============================
# INFERENCE
# ===============================
batch_size = 64
pred_rows = []

with torch.no_grad():
    for i in tqdm(range(0, len(test_ids), batch_size)):
        batch_ids = test_ids[i:i+batch_size]
        idxs = list(range(i, i+len(batch_ids)))

        batch_emb = torch.tensor(test_emb[idxs], dtype=torch.float32).to(DEVICE)

        mf_logits, bp_logits, cc_logits = model(batch_emb)
        
        mf_probs = torch.sigmoid(mf_logits).cpu().numpy()
        bp_probs = torch.sigmoid(bp_logits).cpu().numpy()
        cc_probs = torch.sigmoid(cc_logits).cpu().numpy()

        for j, pid in enumerate(batch_ids):
            # Collect predictions
            preds = []

            # MF
            for k, p in enumerate(mf_probs[j]):
                if p >= thresholds["mf"][k]:
                    preds.append((pid, mf_terms[k], float(p)))

            # BP
            for k, p in enumerate(bp_probs[j]):
                if p >= thresholds["bp"][k]:
                    preds.append((pid, bp_terms[k], float(p)))

            # CC
            for k, p in enumerate(cc_probs[j]):
                if p >= thresholds["cc"][k]:
                    preds.append((pid, cc_terms[k], float(p)))

            # Propagate parents (max rule)
            propagated = {}
            for pid2, go, score in preds:
                for anc in ancestors.get(go, []):
                    propagated[anc] = max(propagated.get(anc, 0), score)

            # Sort by score desc
            final_terms = sorted(propagated.items(), key=lambda x: -x[1])

            # Limit to 1500 terms/protein
            final_terms = final_terms[:1500]

            for go, sc in final_terms:
                pred_rows.append(f"{pid}\t{go}\t{sc:.3f}")


# ===============================
# SAVE SUBMISSION
# ===============================
SUB_PATH = BASE / "submission.tsv"

with open(SUB_PATH, "w") as f:
    for row in pred_rows:
        f.write(row + "\n")

print("[OK] Submission saved:", SUB_PATH)


Loading: /kaggle/working/models/multihead_epoch8_mean0.0038.pth
[OK] Model loaded.
Loading test embeddings...
Test emb shape: (224309, 1024)
Test IDs: 8454


100%|██████████| 133/133 [06:02<00:00,  2.73s/it]

[OK] Submission saved: /kaggle/working/submission.tsv
